In [181]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
from numpy import zeros_like
from IPython.display import display
def display_matrix(m):
    display(sympy.Matrix(m))
import sympy
import pandas as pd
import scipy.stats as ss

sympy.init_printing()

from Pricer import Pricer
from Processes import Processes

from scipy.stats.qmc import Sobol
from scipy.stats import norm

In [182]:
basis = 'leguerre'

# parameters

mu = 0.06
r = 0.06
sigma = 0.04
T = 1
I = 250
paths = 1000
S0 = 37
S0_list = range(37, 43, 1)
m = 0
K = 40
deg = 3

GBM_sigma = sigma
GBM_mu = r

LN_lam = 1
LN_sigma = 0.02
LN_mu = r
LN_v = 0.02

JR_lam = 0.01
JR_sigma = 0.03
JR_mu = r + JR_lam

LNparams = (LN_lam, LN_sigma, LN_mu, LN_v, m)
JRparams = (JR_lam, JR_sigma, JR_mu)
GBMparams = (GBM_mu, GBM_sigma)

# instance
processes = Processes(S0, r, T, K, paths, I, LNparams, JRparams, GBMparams)
pricer = Pricer(S0, r, sigma, T, K, paths, I, LNparams, JRparams, GBMparams)

Processes class created with S0 = 37, paths = 1000, I = 250, T = 1


In [183]:
#GBM_paths = processes.gbm_paths()
#LN_paths = processes.merton_jump_paths()
#JR_paths = processes.merton_jump_to_ruin_paths()

GBM_paths = processes.gbm_paths_antithetic()
LN_paths = processes.merton_jump_paths_antithetic()
JR_paths = processes.merton_jump_to_ruin_paths_antithetic()

In [184]:
V0_GBM, V0_array_GBM, SE_GBM, exercise_matrix_GBM, V_GBM = pricer.LSM(GBM_paths, basis, deg)
V0_LN, V0_array_LN, SE_LN, exercise_matrix_LN, V_LN = pricer.LSM(LN_paths, basis, deg)
V0_JR, V0_array_JR, SE_JR, exercise_matrix_JR, V_JR = pricer.LSM(JR_paths, basis, deg)

# Sampling at expiry

In [185]:
def sampling_at_expiry(type):
    if type=='GBM':
        paths = GBM_paths
        euro_payoff = pricer.closed_formula_GBM(S0, K, T, r, sigma)
        crude_monte_carlo_european = pricer.crude_monte_carlo_european(paths)
        cov = np.cov(crude_monte_carlo_european, V0_array_GBM)
        alpha = -cov[0,1]/cov[0,0]
        adjusted_payoff = V0_GBM + alpha * (crude_monte_carlo_european - np.mean(euro_payoff))
        SE = np.std(adjusted_payoff)/np.sqrt(len(paths))
        return adjusted_payoff, SE
    
    elif type=='LN':
        paths = LN_paths
        euro_payoff = pricer.closed_formula_LN(S0, K, T, r, 10000)
        crude_monte_carlo_european = pricer.crude_monte_carlo_european(paths)
        cov = np.cov(crude_monte_carlo_european, V0_array_LN)
        alpha = -cov[0,1]/cov[0,0]
        adjusted_payoff = V0_LN + alpha * (crude_monte_carlo_european - np.mean(euro_payoff))
        SE = np.std(adjusted_payoff)/np.sqrt(len(paths))
        return adjusted_payoff, SE
    
    else:
        paths = JR_paths
        euro_payoff = pricer.closed_formula_JR(S0, K, T, r, sigma)
        crude_monte_carlo_european = pricer.crude_monte_carlo_european(paths)
        cov = np.cov(crude_monte_carlo_european, V0_array_JR)
        alpha = -cov[0,1]/cov[0,0]
        adjusted_payoff = V0_JR + alpha * (crude_monte_carlo_european - np.mean(euro_payoff))
        SE = np.std(adjusted_payoff)/np.sqrt(len(paths))
        return adjusted_payoff, SE

In [186]:
GBM_sampling_at_expiry, GBM_SE_sampling_at_expiry = sampling_at_expiry('GBM')
LN_sampling_at_expiry, LN_SE_sampling_at_expiry = sampling_at_expiry('LN')
JR_sampling_at_expiry, JR_SE_sampling_at_expiry = sampling_at_expiry('JR')

# Sampling at exercise

In [187]:
def sampling_at_exercise(paths):
    if paths == 'GBM_paths':
        european_put_prices = np.zeros_like(exercise_matrix_GBM, dtype=float)

        for path in range(exercise_matrix_GBM.shape[0]):
            for time in range(exercise_matrix_GBM.shape[1]):
                if exercise_matrix_GBM[path, time]:
                    S_at_exercise = GBM_paths[path, time]
                    t = time * (T / I)  # Current time in years
                    undiscounted_price = pricer.closed_formula_GBM(S_at_exercise, K, T-t, r, sigma)
                    discount_factor = np.exp(-r * t)  # Discount factor back to time 0
                    european_put_prices[path, time] = undiscounted_price * discount_factor

        first_exercise_prices = np.zeros(european_put_prices.shape[0])

        for path in range(european_put_prices.shape[0]):
            # Find the first non-zero price for this path
            first_exercise_price = next((price for price in european_put_prices[path, :] if price > 0), 0)
            first_exercise_prices[path] = first_exercise_price

        crude_monte_carlo_european_GBM = pricer.crude_monte_carlo_european(GBM_paths)

        cov = np.cov(crude_monte_carlo_european_GBM, V0_array_GBM)

        alpha = -cov[0, 1] / cov[0, 0]

        adjusted_payoff = V0_GBM + alpha * (crude_monte_carlo_european_GBM - np.mean(first_exercise_prices))

    elif paths == 'LN_p aths':

        european_put_prices = np.zeros_like(exercise_matrix_LN, dtype=float)

        for path in range(exercise_matrix_LN.shape[0]):
            for time in range(exercise_matrix_LN.shape[1]):
                if exercise_matrix_LN[path, time]:
                    S_at_exercise = LN_paths[path, time]
                    t = time * (T / I)  # Current time in years
                    undiscounted_price = pricer.closed_formula_LN(S_at_exercise, K, T-t, r, 1000000)
                    discount_factor = np.exp(-r * t)  # Discount factor back to time 0
                    european_put_prices[path, time] = undiscounted_price * discount_factor

        first_exercise_prices = np.zeros(european_put_prices.shape[0])

        for path in range(european_put_prices.shape[0]):
            # Find the first non-zero price for this path
            first_exercise_price = next((price for price in european_put_prices[path, :] if price > 0), 0)
            first_exercise_prices[path] = first_exercise_price

        crude_monte_carlo_european = pricer.crude_monte_carlo_european(LN_paths)

        cov = np.cov(crude_monte_carlo_european, V0_array_LN)

        alpha = -cov[0, 1] / cov[0, 0]

        adjusted_payoff = V0_LN + alpha * (crude_monte_carlo_european - np.mean(first_exercise_prices))

    else:
        european_put_prices = np.zeros_like(exercise_matrix_JR, dtype=float)

        for path in range(exercise_matrix_JR.shape[0]):
            for time in range(exercise_matrix_JR.shape[1]):
                if exercise_matrix_JR[path, time]:
                    S_at_exercise = JR_paths[path, time]
                    t = time * (T / I)  # Current time in years
                    undiscounted_price = pricer.closed_formula_JR(S_at_exercise, K, T-t, r, sigma)
                    discount_factor = np.exp(-r * t)  # Discount factor back to time 0
                    european_put_prices[path, time] = undiscounted_price * discount_factor

        first_exercise_prices = np.zeros(european_put_prices.shape[0])

        for path in range(european_put_prices.shape[0]):
            # Find the first non-zero price for this path
            first_exercise_price = next((price for price in european_put_prices[path, :] if price > 0), 0)
            first_exercise_prices[path] = first_exercise_price

        crude_monte_carlo_european = pricer.crude_monte_carlo_european(JR_paths)

        cov = np.cov(crude_monte_carlo_european, V0_array_JR)

        alpha = -cov[0, 1] / cov[0, 0]

        adjusted_payoff = V0_JR + alpha * (crude_monte_carlo_european - np.mean(first_exercise_prices))
    return adjusted_payoff

In [188]:
GBM_sampling_at_exercise = sampling_at_exercise('GBM_paths')
LN_sampling_at_exercise= sampling_at_exercise('LN_paths')
JR_sampling_at_exercise= sampling_at_exercise('JR_paths')

/Users/philipgadereumert/Desktop/Uni/Kandidat/År 1/Seimar Monte Carlo Finance Econometrics/Python/Pricer.py:30: RuntimeWarning: divide by zero encountered in log
  d1 = (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))


In [189]:
GBM_SE_sampling_at_exercise = np.std(GBM_sampling_at_exercise)/np.sqrt(len(GBM_paths))
LN_SE_sampling_at_exercise = np.std(LN_sampling_at_exercise)/np.sqrt(len(LN_paths))
JR_SE_sampling_at_exercise = np.std(JR_sampling_at_exercise)/np.sqrt(len(JR_paths))

In [190]:
print('GBM')
print('sampling at expiry:', GBM_SE_sampling_at_expiry)
print('sampling at exercise: ', GBM_SE_sampling_at_exercise)
print('LSM:' , SE_GBM)

print('LN')
print('sampling at expiry: ', LN_SE_sampling_at_expiry)
print('sampling at exercise: ', LN_SE_sampling_at_exercise)
print('LSM:' , SE_LN)

print('JR')
print('sampling at expiry: ', JR_SE_sampling_at_expiry)
print('sampling at exercise: ', JR_SE_sampling_at_exercise)
print('LSM:' , SE_JR)

GBM
sampling at expiry: 0.0001106874039731872
sampling at exercise:  0.00011068740397318773
LSM: 0.0020898516072104735
LN
sampling at expiry:  4.6789495264781194e-05
sampling at exercise:  4.678949526478118e-05
LSM: 0.0013044128978395133
JR
sampling at expiry:  0.005755132357228153
sampling at exercise:  0.005755132357228154
LSM: 0.02623827768864536


In [191]:
print('GBM')
print('sampling at expiry: ', GBM_SE_sampling_at_expiry)
print('sampling at exercise: ', GBM_SE_sampling_at_exercise)
print('LSM:' , SE_GBM)

print('LN')
print('sampling at expiry: ', LN_SE_sampling_at_expiry)
print('sampling at exercise: ', LN_SE_sampling_at_exercise)
print('LSM:' , SE_LN)

print('JR')
print('sampling at expiry: ', JR_SE_sampling_at_expiry)
print('sampling at exercise: ', JR_SE_sampling_at_exercise)
print('LSM:' , SE_JR)

GBM
sampling at expiry:  0.0001106874039731872
sampling at exercise:  0.00011068740397318773
LSM: 0.0020898516072104735
LN
sampling at expiry:  4.6789495264781194e-05
sampling at exercise:  4.678949526478118e-05
LSM: 0.0013044128978395133
JR
sampling at expiry:  0.005755132357228153
sampling at exercise:  0.005755132357228154
LSM: 0.02623827768864536
